In [ ]:
import os
import sys
import torch
import gpytorch
from tqdm.auto import tqdm
import timeit

if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

In [ ]:
from bi_gp.bilateral_kernel import BilateralKernel

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [ ]:
N_vals = torch.linspace(100, 10000000, 10).int().tolist()
D_vals = torch.linspace(1, 100, 10).int().tolist()

# Matmul

In [ ]:
N_vary = []
for N in tqdm(N_vals):
    D = 1
    x = torch.randn(N, D).to(device)
    K = BilateralKernel().to(device)(x)
    v = torch.randn(N, 1).to(device)
    def matmul():
        return K @ v
    time = timeit.timeit(matmul , number=10)

    N_vary.append([N, D, time])

    del x
    del K
    del v
    del matmul

In [ ]:
D_vary = []
for D in tqdm(D_vals):
    N = 1000
    x = torch.randn(N, D).to(device)
    K = BilateralKernel().to(device)(x)
    v = torch.randn(N, 1).to(device)
    def matmul():
        return K @ v
    time = timeit.timeit(matmul , number=10)

    D_vary.append([N, D, time])

    del x
    del K
    del v
    del matmul

In [ ]:
import pandas as pd
N_vary = pd.DataFrame(N_vary, columns=["N", "D", "Time"])
D_vary = pd.DataFrame(D_vary, columns=["N", "D", "Time"])

In [ ]:
import seaborn as sns
ax = sns.lineplot(data=N_vary, x="N", y="Time")
ax.set(title="Matmul (D=1)")

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
regr = LinearRegression()
regr.fit(np.log(D_vary["D"].to_numpy()[:, None]), np.log(D_vary["Time"]))
print('Coefficients: \n', regr.coef_)
pred_time = regr.predict(np.log(D_vary["D"].to_numpy()[:, None]))

ax = sns.lineplot(data=D_vary, x="D", y="Time")
ax.set(title="Matmul (N=1000)", xscale="log", yscale="log")
ax.plot(D_vary["D"].to_numpy(), np.exp(pred_time))

# Gradient

In [ ]:
N_vary = []
for N in tqdm(N_vals):
    D = 1
    x = torch.randn(N, D, requires_grad=True).to(device)
    K = BilateralKernel().to(device)(x)
    v = torch.randn(N, 1, requires_grad=True).to(device)
    sum = (K @ v).sum()
    def gradient():
        torch.autograd.grad(sum, [x, v], retain_graph=True)
        x.grad = None
        v.grad = None
        return
    time = timeit.timeit(gradient, number=10)

    N_vary.append([N, D, time])

    del x
    del K
    del v
    del gradient

In [ ]:
D_vary = []
for D in tqdm(D_vals):
    N = 1000
    x = torch.randn(N, D, requires_grad=True).to(device)
    K = BilateralKernel().to(device)(x)
    v = torch.randn(N, 1, requires_grad=True).to(device)
    sum = (K @ v).sum()
    def gradient():
        torch.autograd.grad(sum, [x, v], retain_graph=True)
        x.grad = None
        v.grad = None
        return
    time = timeit.timeit(gradient, number=10)

    D_vary.append([N, D, time])

    del x
    del K
    del v
    del gradient

In [ ]:
import pandas as pd
N_vary = pd.DataFrame(N_vary, columns=["N", "D", "Time"])
D_vary = pd.DataFrame(D_vary, columns=["N", "D", "Time"])

In [ ]:
import seaborn as sns
ax = sns.lineplot(data=N_vary, x="N", y="Time")
ax.set(title="Gradient computation of (K@v).sum() (D=1)")

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
regr = LinearRegression()
regr.fit(np.log(D_vary["D"].to_numpy()[:, None]), np.log(D_vary["Time"]))
print('Coefficients: \n', regr.coef_)
pred_time = regr.predict(np.log(D_vary["D"].to_numpy()[:, None]))

ax = sns.lineplot(data=D_vary, x="D", y="Time")
ax.set(title="Gradient computation of (K@v).sum() (N=100)", xscale="log", yscale="log")
ax.plot(D_vary["D"].to_numpy(), np.exp(pred_time))